In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path

sys.path.insert(0, "/home/saxelrod/Repo/projects/dimenet_nff/NeuralForceField")

import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader

from nff.data import Dataset, split_train_validation_test, collate_dicts, to_tensor
from nff.train import Trainer, get_trainer, get_model, load_model, loss, hooks, metrics, evaluate

In [3]:
np.random.seed(0)

num_atoms = 6
z = torch.Tensor(np.random.randint(1, 12, num_atoms))
xyz = torch.Tensor(np.random.rand(num_atoms, 3))
nxyz = torch.cat((z.reshape(-1, 1), xyz), dim=-1)
nxyz

tensor([[ 6.0000,  0.6236,  0.3844,  0.2975],
        [ 1.0000,  0.0567,  0.2727,  0.4777],
        [ 4.0000,  0.8122,  0.4800,  0.3928],
        [ 4.0000,  0.8361,  0.3374,  0.6482],
        [ 8.0000,  0.3682,  0.9572,  0.1404],
        [10.0000,  0.8701,  0.4736,  0.8009]])

In [4]:
props = {"nxyz": [nxyz], "energy": torch.Tensor([1.3])}
dataset = Dataset(props)

In [5]:
import pdb

cutoff = 5.0
dataset.generate_neighbor_list(cutoff, undirected=False)
_ = dataset.generate_angle_list()


In [6]:
dataset.props['angle_list']

[tensor([[0, 1, 2],
         [0, 1, 3],
         [0, 1, 4],
         [0, 1, 5],
         [0, 2, 1],
         [0, 2, 3],
         [0, 2, 4],
         [0, 2, 5],
         [0, 3, 1],
         [0, 3, 2],
         [0, 3, 4],
         [0, 3, 5],
         [0, 4, 1],
         [0, 4, 2],
         [0, 4, 3],
         [0, 4, 5],
         [0, 5, 1],
         [0, 5, 2],
         [0, 5, 3],
         [0, 5, 4],
         [1, 0, 2],
         [1, 0, 3],
         [1, 0, 4],
         [1, 0, 5],
         [1, 2, 0],
         [1, 2, 3],
         [1, 2, 4],
         [1, 2, 5],
         [1, 3, 0],
         [1, 3, 2],
         [1, 3, 4],
         [1, 3, 5],
         [1, 4, 0],
         [1, 4, 2],
         [1, 4, 3],
         [1, 4, 5],
         [1, 5, 0],
         [1, 5, 2],
         [1, 5, 3],
         [1, 5, 4],
         [2, 0, 1],
         [2, 0, 3],
         [2, 0, 4],
         [2, 0, 5],
         [2, 1, 0],
         [2, 1, 3],
         [2, 1, 4],
         [2, 1, 5],
         [2, 3, 0],
         [2, 3, 1],


In [29]:
import pdb

modelparams = {"n_rbf": 6,
               "cutoff": 5.0,
               "envelope_p": 6,
               "n_spher": 6,
               "l_spher": 7,
               "atom_embed_dim": 128,
               "n_bilinear": 8,
               "activation": "swish",
               "n_convolutions": 6,
               "output_keys": ["energy"],
               "grad_keys": ["energy_grad"]}

try:
    model = get_model(modelparams, model_type="DimeNet")
except Exception as e:
    print(e)
    pdb.post_mortem()

In [35]:
batch = {"nxyz": dataset.props["nxyz"][0],
        "energy": dataset.props["energy"],
        "nbr_list": dataset.props["nbr_list"][0],
        "angle_list": dataset.props["angle_list"][0],
        "num_atoms": torch.LongTensor([len(dataset.props["nxyz"][0])])}

In [37]:
import pdb
from nff.utils.cuda import batch_to

device = 0
model.to(device)
batch = batch_to(batch, device)

try:
    out = model(batch)
except Exception as e:
    print(e)
    pdb.post_mortem()

In [40]:
model(batch)

{'energy': tensor([65956.2188], device='cuda:0', grad_fn=<StackBackward>),
 'energy_grad': tensor([[ 1.4452e+04,  7.5718e+03,  7.8126e+03],
         [ 2.4429e+02,  3.6896e+01, -1.5352e+01],
         [ 8.0630e+03, -2.4977e+04,  1.6034e+05],
         [ 1.6894e+04,  9.0413e+04,  5.8000e+04],
         [-7.8009e+01,  2.0182e+02,  1.2939e+00],
         [-3.9575e+04, -7.3247e+04, -2.2614e+05]], device='cuda:0',
        grad_fn=<AddBackward0>)}